In [2]:
import geopandas as gpd
import pandas as pd

In [6]:
zip_codes = gpd.read_file("../../data/geospacial/Zip_Codes/Zip_Codes.shp")
neighborhood_clusters = gpd.read_file("../../data/geospacial/Neighborhood_Clusters/Neighborhood_Clusters.shp")

### Map neighborhood clusters to zip codes

In [7]:
intersection = gpd.overlay(neighborhood_clusters, zip_codes, how='intersection') # get the intersection of zip codes and neighborhood clusters
intersection['area'] = intersection.geometry.area # add column to get areas of each intersection

In [8]:
matches = intersection.groupby('NAME_1').apply(lambda x: x.loc[x['area'].idxmax()]).reset_index(drop=True) # keep the column for each niehborhood cluster with the greatest area (the zip code that most fully contains the nieghborhood cluster)

/var/folders/5c/d2k1t24s5zl213152tpfh6nh0000gn/T/ipykernel_38571/3276486303.py:1: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  matches = intersection.groupby('NAME_1').apply(lambda x: x.loc[x['area'].idxmax()]).reset_index(drop=True) # keep the column for each niehborhood cluster with the greatest area (the zip code that most fully contains the nieghborhood cluster)


In [9]:
cluster_to_zip = matches[["NAME_1","ZIPCODE"]] # the mapping! note that one zip code might contain several clusters 
cluster_to_zip.head()

,NAME_1,ZIPCODE
0,Cluster 1,20009
1,Cluster 10,20015
2,Cluster 11,20016
3,Cluster 12,20008
4,Cluster 13,20016


### Map Neighborhoods to clusters

In [10]:
neighborhood_to_cluster = neighborhood_clusters[["NAME","NBH_NAMES"]] # Grab clusters and associated neighborhoods

In [11]:
neighborhood_to_cluster["NBH_NAMES"] = [x.split(", ") for x in neighborhood_to_cluster["NBH_NAMES"]] # split into lists

/var/folders/5c/d2k1t24s5zl213152tpfh6nh0000gn/T/ipykernel_38571/3712776356.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  neighborhood_to_cluster["NBH_NAMES"] = [x.split(", ") for x in neighborhood_to_cluster["NBH_NAMES"]] # split into lists


In [12]:
neighborhood_to_cluster = neighborhood_to_cluster.explode("NBH_NAMES") # explode
neighborhood_to_cluster.head()

,NAME,NBH_NAMES
0,Cluster 16,Colonial Village
0,Cluster 16,Shepherd Park
0,Cluster 16,North Portal Estates
1,Cluster 41,Rock Creek Park
2,Cluster 10,Hawthorne


### Join

In [13]:
joined = pd.merge(neighborhood_to_cluster,cluster_to_zip,left_on="NAME", right_on="NAME_1",how="right")
joined = joined.drop("NAME_1", axis=1)

### Save!

In [14]:
joined.to_csv("../../data/geospacial/geo_matching.csv",index=False)